# Chapter 14: TensorBoard

Bab ini membahas penggunaan TensorBoard sebagai alat visualisasi utama dalam ekosistem TensorFlow untuk memantau metrik, menganalisis performa *pipeline*, dan melakukan profil sistem guna mendeteksi hambatan (*bottleneck*).

---

## 14.1 Visualisasi Data & Inisialisasi
TensorBoard memungkinkan kita melihat data (gambar, teks, atau audio) sebelum dan selama pelatihan untuk memastikan *preprocessing* berjalan benar.
* **Mekanisme**: Menggunakan `tf.summary.create_file_writer()` untuk menentukan lokasi penyimpanan log.
* **Eksekusi di Colab**: Gunakan perintah ajaib `%load_ext tensorboard` diikuti dengan `%tensorboard --logdir logs` untuk memunculkan dasbor langsung di dalam notebook.

---

## 14.2 Melacak & Monitoring Model
Selama proses `model.compile()`, kita menentukan *loss function* (seperti `SparseCategoricalCrossentropy`) dan optimizer (seperti `Adam`).
* **Scalars**: TensorBoard akan merekam nilai *loss* dan *accuracy* pada setiap epoch secara otomatis jika kita menambahkan `tf.keras.callbacks.TensorBoard` ke dalam fungsi `fit()`.
* **Graph**: Memungkinkan kita melihat struktur arsitektur model secara visual untuk memastikan koneksi antar layer sudah benar.



---

## 14.3 Profiling untuk Deteksi Bottleneck
Seringkali pelatihan berjalan lambat bukan karena GPU yang lemah, melainkan karena aliran data yang terhambat (*bottleneck*).
* **Tooling**: Memerlukan paket `tensorboard_plugin_profile` dan pustaka sistem `libcupti`.
* **Analisis Device**: Dasbor profil akan memperlihatkan waktu komputasi di setiap perangkat (CPU vs GPU). Jika CPU bekerja 100% sementara GPU sering menganggur (*idle*), berarti terjadi masalah pada *input pipeline*.

---

## 14.4 Mengoptimasi Pipeline Input
Untuk mengatasi *bottleneck* yang terdeteksi, TensorFlow menyediakan teknik optimasi pada `tf.data`:
* **Data Prefetching**: Menggunakan `.prefetch(buffer_size=tf.data.AUTOTUNE)` agar CPU menyiapkan *batch* berikutnya selagi GPU sedang memproses *batch* saat ini.
* **Parallelized Map**: Menggunakan argumen `num_parallel_calls` pada fungsi `.map()` untuk mempercepat transformasi data menggunakan banyak core CPU.
* **TF_GPU_THREAD_MODE**: Variabel lingkungan untuk mengoptimasi cara *thread* GPU berkomunikasi dengan CPU guna meminimalkan latensi.



---

## 14.5 Mixed Precision Training
Teknik untuk mempercepat pelatihan dan mengurangi penggunaan memori tanpa kehilangan banyak akurasi.
* **Metode**: Menggunakan kombinasi tipe data **16-bit float** (lebih cepat, hemat memori) untuk komputasi dan **32-bit float** (lebih akurat) untuk penyimpanan bobot yang krusial.
* **Implementasi**: Cukup dengan mengatur `mixed_precision.set_global_policy('mixed_float16')`. TensorBoard menyediakan *Memory Profile* untuk melihat seberapa besar penghematan memori yang dihasilkan.

---

## 14.6 Visualizing Word Vectors (Embedding Projector)
TensorBoard memiliki fitur **Projector** untuk memvisualisasikan data berdimensi tinggi (seperti Word Embeddings) ke dalam ruang 3D yang bisa diputar.
* **GloVe (Global Vectors)**: Sering digunakan sebagai *pretrained word vectors* untuk mengurangi beban pelatihan. GloVe menggabungkan statistik kemunculan kata global dan lokal dalam sebuah korpus.
* **Analisis**: Kita bisa mencari kata-kata yang secara visual berdekatan dalam grafik 3D untuk memverifikasi apakah model memahami hubungan semantik (misal: "Jakarta" berdekatan dengan "Indonesia").



---

## 14.7 Tip Praktis di Colab
Untuk menggunakan Profiler secara maksimal, pastikan kamu menyalakan fitur profil pada callback:

```python
import datetime

# Membuat folder log unik berdasarkan waktu
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Menyiapkan callback TensorBoard dengan fitur profile
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    profile_batch='500, 520' # Melakukan profil pada batch ke 500 sampai 520
)

# Jalankan pelatihan dengan callback tersebut
# model.fit(train_data, epochs=5, callbacks=[tensorboard_callback])